## Introduction:
This notebook describes steps for Loading a small text dataset, augmenting it, and applying preprocessing techniques.
Finally fine-tuning a GPT-Neo 1.3 B parameter model using Deep Speed libary and inference.

In [1]:
print('importing libraries')
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import warnings 
import time
import math
import pandas as pd
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

from transformers import AutoTokenizer, AutoModelForCausalLM
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from tqdm import tqdm
tqdm.pandas()

from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AdamW, get_linear_schedule_with_warmup


importing libraries


2023-08-29 21:34:35.291007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 21:34:35.411347: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-29 21:34:35.804273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-29 21:34:35.804324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

This file contains custom function...


## Augmenting the dataset through paraphrasing.
In this section, we will be expanding our dataset by paraphrasing it using a pre-train **pegasus_paraphrase** model.

In [6]:
!pip install spacy
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
2023-08-29 21:12:20.439130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 21:12:20.558270: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-29 21:12:20.974335: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-29 21:12:20.9

In [9]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [10]:
import pandas as pd
df = pd.read_csv("tipsDataset2.txt", sep="\t", header=None, names=["text"])

In [11]:
# existing small dataset
df

,text
0,<HAPPY> Spread your happiness by sharing a gen...
1,"<HAPPY> Dance like nobody's watching, and let ..."
2,<HAPPY> Write down three things you're gratefu...
3,<HAPPY> Engage in a favorite hobby that brings...
4,<HAPPY> Surround yourself with people who upli...
...,...
2105,<SURPRISED> Reflect on how the surprise enhanc...
2106,"<ANGRY> Anger is a sign of weakness, not stren..."
2107,<ANGRY> Anger is like a fire that burns everyt...
2108,<ANGRY> Anger is a poison that you drink and e...


## PREPROCESSING

In [12]:
# split the text column by < and >
df[['tag', 'space', 'text']] = df['text'].str.split('<|>', expand=True)
# drop the space column
df = df.drop('tag', axis=1)
df

,text,space
0,Spread your happiness by sharing a genuine sm...,HAPPY
1,"Dance like nobody's watching, and let your jo...",HAPPY
2,Write down three things you're grateful for e...,HAPPY
3,Engage in a favorite hobby that brings you pu...,HAPPY
4,Surround yourself with people who uplift your...,HAPPY
...,...,...
2105,Reflect on how the surprise enhances your sen...,SURPRISED
2106,"Anger is a sign of weakness, not strength. Th...",ANGRY
2107,Anger is like a fire that burns everything in...,ANGRY
2108,Anger is a poison that you drink and expect t...,ANGRY


## AUGMENTING

In [1]:
def get_response(row):
    """
    Generate multiple responses to an input text using a transformer-based language model.

    This function takes an input text, tokenizes it, and generates multiple responses using
    a transformer-based language model. It returns a list of generated responses.

    Parameters:
        row (pandas.Series): A pandas Series containing the 'text' field as the input text.

    Returns:
        list: A list of generated responses as strings.
    """
    input_text = row['text']

    # Tokenize the input text and prepare it for generation
    batch = tokenizer([input_text], truncation=True, padding='longest', max_length=60, return_tensors="pt").to(torch_device)

    # Generate multiple responses using the language model
    translated = model.generate(**batch, max_length=60, num_beams=3, num_return_sequences=3, temperature=1.0)

    # Decode the generated responses and skip special tokens
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return tgt_text

In [17]:
df3 = df.copy()
df3

,text,space
0,Spread your happiness by sharing a genuine sm...,HAPPY
1,"Dance like nobody's watching, and let your jo...",HAPPY
2,Write down three things you're grateful for e...,HAPPY
3,Engage in a favorite hobby that brings you pu...,HAPPY
4,Surround yourself with people who uplift your...,HAPPY
...,...,...
2105,Reflect on how the surprise enhances your sen...,SURPRISED
2106,"Anger is a sign of weakness, not strength. Th...",ANGRY
2107,Anger is like a fire that burns everything in...,ANGRY
2108,Anger is a poison that you drink and expect t...,ANGRY


In [18]:
df3['text'] = df.progress_apply(lambda row: get_response(row), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2110/2110 [05:48<00:00,  6.05it/s]


In [19]:
df_exploded = df3.explode('text')

In [20]:
df_final= pd.concat([df_exploded,df],axis=0)

In [21]:
df_final

,text,space
0,Sharing a genuine smile with someone will spre...,HAPPY
0,Share a genuine smile with someone.,HAPPY
0,Sharing a genuine smile with someone will make...,HAPPY
1,"Let your joy shine, and dance like nobody's wa...",HAPPY
1,Let your joy shine and dance like nobody's wat...,HAPPY
...,...,...
2105,Reflect on how the surprise enhances your sen...,SURPRISED
2106,"Anger is a sign of weakness, not strength. Th...",ANGRY
2107,Anger is like a fire that burns everything in...,ANGRY
2108,Anger is a poison that you drink and expect t...,ANGRY


In [22]:
df_final.to_csv('tips_augmented.csv',index=False)

In [3]:
df_final = pd.read_csv('tips_augmented.csv')

In [4]:
df_final

,text,space
0,Sharing a genuine smile with someone will spre...,HAPPY
1,Share a genuine smile with someone.,HAPPY
2,Sharing a genuine smile with someone will make...,HAPPY
3,"Let your joy shine, and dance like nobody's wa...",HAPPY
4,Let your joy shine and dance like nobody's wat...,HAPPY
...,...,...
8435,Reflect on how the surprise enhances your sen...,SURPRISED
8436,"Anger is a sign of weakness, not strength. Th...",ANGRY
8437,Anger is like a fire that burns everything in...,ANGRY
8438,Anger is a poison that you drink and expect t...,ANGRY


## FINE TUNING GPT-NEO USING DEEPSPEED LIBRARY
Pompt based fine-tuning of the pre-trained GPT-NEO model is done.


In [6]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B",pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50258, 2048)

In [7]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=50, save_strategy="epoch",
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1, warmup_steps=50,
                                  weight_decay=0.01, logging_dir='./logs', fp16=True, deepspeed='./ds_config_gpt_neo_27-Copy1.json')


[2023-08-29 21:35:33,592] [INFO] [comm.py:617:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-08-29 21:35:34,154] [INFO] [comm.py:669:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.28.177.139, master_port=29500
[2023-08-29 21:35:34,155] [INFO] [comm.py:633:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [8]:
descriptions = df_final['text']
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
print("Max length: {}".format(max_length))
print(type(max_length))
if max_length > 2048:
    max_length = 2048

print("New Max length: {}".format(max_length))

class custom_Dataset(Dataset):
    """
    Custom PyTorch dataset for text and emotion data.

    This dataset class takes a list of text inputs and their corresponding emotions,
    tokenizes the data using a tokenizer, and prepares it for use in a PyTorch model.

    Parameters:
        txt_list (list): A list of text inputs.
        emotion (list): A list of corresponding emotions for each text input.
        tokenizer: Tokenizer for processing text inputs.
        max_len (int): The maximum sequence length for tokenized inputs.

    Returns:
        tuple: A tuple containing input_ids and attention_masks as tensors.
    """    
    def __init__(self, txt_list,emotion, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = txt_list
        self.max_len = max_len
        self.input_ids = []
        self.attn_masks = []

        for txt,emo in zip(txt_list,emotion):
            if emo == 'HAPPY':
                prompt  = f'<startoftext>HAPPY:{txt}'
            elif  emo == 'SAD':
                prompt  = f'<startoftext>SAD:{txt}'
            elif  emo == 'SURPRISED':
                prompt  = f'<startoftext>SURPRISED:{txt}'
            elif  emo == 'CALM':
                prompt  = f'<startoftext>CALM:{txt}'
            elif  emo == 'ANGRY':
                prompt  = f'<startoftext>ANGRY:{txt}'
            elif  emo == 'DISGUSTED':
                prompt  = f'<startoftext>DISGUSTED:{txt}'
            elif  emo == 'FEARFUL':
                prompt  = f'<startoftext>FEARFUL:{txt}'

            encodings_dict = tokenizer(prompt + '<|endoftext|>', truncation=True,
                                       max_length=max_len, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))                  

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]


Max length: 58
<class 'int'>
New Max length: 58


In [9]:
dataset = custom_Dataset(df_final['text'],df_final['space'], tokenizer, max_len=max_length)
print(f'dataset len is {len(dataset)}')
train_size = int(0.9 * len(dataset))
print(f'train_size is {train_size}')
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

dataset len is 8440
train_size is 7596


In [10]:
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()




Using cuda_amp half precision backend


[2023-08-29 21:35:40,230] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.7.3, git-hash=unknown, git-branch=unknown


2023-08-29 21:35:40 | INFO | torch.distributed.distributed_c10d | Added key: store_based_barrier_key:2 to store for rank: 0
2023-08-29 21:35:40 | INFO | torch.distributed.distributed_c10d | Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.


[2023-08-29 21:35:40,670] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-08-29 21:35:41,802] [WARNING] [cpu_adam.py:83:__init__] FP16 params for CPUAdam may not work on AMD CPUs


Using /home/varun/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/varun/.cache/torch_extensions/py310_cu117/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...


...

[2023-08-29 21:35:58,377] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 4294967296, reducing to 2147483648.0
[2023-08-29 21:35:58,755] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 2147483648.0, reducing to 1073741824.0
[2023-08-29 21:35:59,120] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 1073741824.0, reducing to 536870912.0
[2023-08-29 21:35:59,499] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 536870912.0, reducing to 268435456.0
[2023-08-29 21:35:59,866] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 268435456.0, reducing to 134217728.0
[2023-08-29 21:36:00,244] [INFO] [stage_1_and_2.py:1720:step] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 134217728.0, reducing to 67108864.0
[2023-08

Saving model checkpoint to ./results/checkpoint-7596
Configuration saved in ./results/checkpoint-7596/config.json
Model weights saved in ./results/checkpoint-7596/pytorch_model.bin


[2023-08-29 23:25:44,538] [INFO] [logging.py:68:log_dist] [Rank 0] [Torch] Checkpoint global_step7596 is begin to save!
[2023-08-29 23:25:44,541] [INFO] [logging.py:68:log_dist] [Rank 0] Saving model checkpoint: ./results/checkpoint-7596/global_step7596/mp_rank_00_model_states.pt
[2023-08-29 23:25:44,541] [INFO] [torch_checkpoint_engine.py:15:save] [Torch] Saving ./results/checkpoint-7596/global_step7596/mp_rank_00_model_states.pt...
[2023-08-29 23:25:47,075] [INFO] [torch_checkpoint_engine.py:17:save] [Torch] Saved ./results/checkpoint-7596/global_step7596/mp_rank_00_model_states.pt.
[2023-08-29 23:25:47,076] [INFO] [torch_checkpoint_engine.py:15:save] [Torch] Saving ./results/checkpoint-7596/global_step7596/zero_pp_rank_0_mp_rank_00_optim_states.pt...
[2023-08-29 23:25:57,659] [INFO] [torch_checkpoint_engine.py:17:save] [Torch] Saved ./results/checkpoint-7596/global_step7596/zero_pp_rank_0_mp_rank_00_optim_states.pt.
[2023-08-29 23:25:57,661] [INFO] [engine.py:3196:_save_zero_checkpo



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7596, training_loss=0.5336930856006656, metrics={'train_runtime': 6606.6123, 'train_samples_per_second': 1.15, 'train_steps_per_second': 1.15, 'total_flos': 3194440023801856.0, 'train_loss': 0.5336930856006656, 'epoch': 1.0})

## STORING THE TRAINED MODEL

In [11]:
tokenizer.save_pretrained('./gpt1_3B_deepspeed_tokenizer/')
model.save_pretrained('./gpt1_3B_deepspeed_model')

tokenizer config file saved in ./gpt1_3B_deepspeed_tokenizer/tokenizer_config.json
Special tokens file saved in ./gpt1_3B_deepspeed_tokenizer/special_tokens_map.json
Configuration saved in ./gpt1_3B_deepspeed_model/config.json
Model weights saved in ./gpt1_3B_deepspeed_model/pytorch_model.bin


## LOADING AND INFERENCE

In [12]:
tokenizer = AutoTokenizer.from_pretrained('./gpt1_3B_deepspeed_tokenizer/', pad_token='<|pad|>',bos_token='<|startoftext|>',eos_token='<|endoftext|>')
model = AutoModelForCausalLM.from_pretrained('./gpt1_3B_deepspeed_model').cuda()

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file ./gpt1_3B_deepspeed_model/config.json
Model config GPTNeoConfig {
  "_name_or_path": "./gpt1_3B_deepspeed_model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      12
    ]
  

In [13]:
input_sentence=""
outputs = []
prompt  = f'<startoftext>SAD:{input_sentence}'
generated = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=30, top_p=0.95, temperature=1.9, num_return_sequences=10)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True).split(':')[-1])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: <startoftext>SAD:You can learn a new activity in a virtual skill session when you reach the exhilarating Surprise at your day time
1: <startoftext>SAD:Explore hobbies about playing in a virtual playing day.
2: <startoftext>SAD:Cultivate a sense of growth within relationships who encourage emotional discovery.
3: <startoftext>SAD:Accept in some spiritual texts.
4: <startoftext>SAD: Take up potteryoamining as a project within for your community's benefit.
5: <startoftext>SAD:Seek out the humor.
6: <startoftext>SAD:Explore the grief community.
7: <startoftext>SAD:To reach up and improve mental health, make.
8: <startoftext>SAD: Try arts therapy for healing, like ceramics and artistic quotes.
9: <startoftext>SAD:Time each day for bed a deep breathing exercises: stretching before bed by enjoying each achievement and reflecting at the events


In [15]:
input_sentence=""
outputs = []
prompt  = f'<startoftext>ANGRY:{input_sentence}'
generated = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=30, top_p=0.95, temperature=1.9, num_return_sequences=10)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True).split(':')[-1])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: <startoftext>ANGRY:A positive physical imagery can be used in a mantra like, future for addressing anger assertively face assert your
1: <startoftext>ANGRY: Reflect on the physical sensations anger managed using physical activities.
2: <startoftext>ANGRY:Imagine that overcoming even challenging goals you'll reach after addressing anger-related situations.
3: <startoftext>ANGRY:Accept it difficult time.
4: <startoftext>ANGRY:Time to disengage from physical symptoms is an activity the can be done within with practicing forgiveness.
5: <startoftext>ANGRY:Time-outs remind you that stress-based training challenges will not impact your relationships.
6: <startoftext>ANGRY:Communication skills training can be learned and learned in conflicts.
7: <startoftext>ANGRY:Write letters through a message you should leave a calming environment.
8: <startoftext>ANGRY:If anger has consequences consequences that reflect on your past feelings of responsibility, consider action.
9: <startoftext>ANGRY: Us

----- end of notebook